<a href="https://colab.research.google.com/github/seemanth-raju/LungCancerPrediction/blob/main/Welcome_to_Colaboratory.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!git clone https://github.com/charlesq34/pointnet.git



Cloning into 'pointnet'...
remote: Enumerating objects: 235, done.
remote: Total 235 (delta 0), reused 0 (delta 0), pack-reused 235
Receiving objects: 100% (235/235), 531.28 KiB | 1.14 MiB/s, done.
Resolving deltas: 100% (133/133), done.


In [2]:
!cd pointnet


ERROR: Could not open requirements file: [Errno 2] No such file or directory: 'requirements.txt'


In [3]:
!pip install tensorflow opencv-python


In [62]:
!pip install trimesh pygltflib


In [14]:
import trimesh
import numpy as np

# Load the GLB file as a scene
scene = trimesh.load('/content/coco_cola_can.glb')

# If the scene contains multiple geometries, you can merge them into a single mesh
mesh = trimesh.util.concatenate(scene.dump()) if isinstance(scene, trimesh.Scene) else scene

# Sample points from the surface of the mesh
point_cloud = mesh.sample(1024)  # Sample 1024 points


In [15]:
import torch
from torch.utils.data import Dataset, DataLoader

class PointCloudDataset(Dataset):
    def __init__(self, point_cloud, label):
        self.point_cloud = point_cloud
        self.label = label

    def __len__(self):
        return 1  # Only one sample

    def __getitem__(self, idx):
        return torch.tensor(self.point_cloud, dtype=torch.float32), torch.tensor(self.label, dtype=torch.long)

# Create dataset and dataloader
label = 0  # Assuming '0' is the label for Coca-Cola can
dataset = PointCloudDataset(point_cloud, label)
dataloader = DataLoader(dataset, batch_size=1, shuffle=True)


In [16]:
import torch
import torch.nn as nn
from torchvision.models import resnet18

class SimplePointNet(nn.Module):
    def __init__(self, num_classes=1):
        super(SimplePointNet, self).__init__()
        self.feature_extractor = resnet18(pretrained=True)
        self.feature_extractor.fc = nn.Identity()  # Remove the final classification layer

        self.fc = nn.Sequential(
            nn.Linear(512, 256),
            nn.ReLU(),
            nn.Dropout(0.3),
            nn.Linear(256, num_classes)
        )

    def forward(self, x):
        # Assuming input x is of shape (B, N, 3) where B is batch size and N is the number of points
        x = x.permute(0, 2, 1)  # Reshape to (B, 3, N)
        x = self.feature_extractor(x)
        x = self.fc(x)
        return x

# Instantiate and load the model
model = SimplePointNet(num_classes=2)  # Assuming binary classification for now
model = model.to('cuda' if torch.cuda.is_available() else 'cpu')


/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet18_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet18_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
Downloading: "https://download.pytorch.org/models/resnet18-f37072fd.pth" to /root/.cache/torch/hub/checkpoints/resnet18-f37072fd.pth
100%|██████████| 44.7M/44.7M [00:00<00:00, 185MB/s]


In [17]:
import torch.optim as optim

# Loss and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

# Training loop
model.train()
for epoch in range(10):  # Run for 10 epochs
    for points, labels in dataloader:
        points, labels = points.to('cuda'), labels.to('cuda')

        # Zero the parameter gradients
        optimizer.zero_grad()

        # Forward pass
        outputs = model(points)
        loss = criterion(outputs, labels)

        # Backward pass and optimize
        loss.backward()
        optimizer.step()

    print(f'Epoch [{epoch+1}/10], Loss: {loss.item():.4f}')

# Evaluation
model.eval()
with torch.no_grad():
    for points, labels in dataloader:
        points, labels = points.to('cuda'), labels.to('cuda')
        outputs = model(points)
        _, predicted = torch.max(outputs, 1)
        print(f'Predicted label: {predicted.item()}, True label: {labels.item()}')


RuntimeError: Given groups=1, weight of size [64, 3, 7, 7], expected input[1, 1, 3, 1024] to have 3 channels, but got 1 channels instead

In [18]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
import numpy as np

# PointNet implementation
class SimplePointNet(nn.Module):
    def __init__(self, num_classes=2):
        super(SimplePointNet, self).__init__()
        self.conv1 = nn.Conv1d(3, 64, 1)
        self.conv2 = nn.Conv1d(64, 128, 1)
        self.conv3 = nn.Conv1d(128, 1024, 1)
        self.fc1 = nn.Linear(1024, 512)
        self.fc2 = nn.Linear(512, 256)
        self.fc3 = nn.Linear(256, num_classes)
        self.relu = nn.ReLU()
        self.dropout = nn.Dropout(p=0.3)

    def forward(self, x):
        x = x.transpose(2, 1)  # Transpose to shape (B, 3, N)
        x = self.relu(self.conv1(x))
        x = self.relu(self.conv2(x))
        x = self.relu(self.conv3(x))
        x = torch.max(x, 2, keepdim=True)[0]  # Max pooling
        x = x.view(-1, 1024)
        x = self.relu(self.fc1(x))
        x = self.dropout(x)
        x = self.relu(self.fc2(x))
        x = self.fc3(x)
        return x

# Create the dataset class
class PointCloudDataset(Dataset):
    def __init__(self, point_cloud, label):
        self.point_cloud = point_cloud
        self.label = label

    def __len__(self):
        return 1  # Only one sample

    def __getitem__(self, idx):
        return torch.tensor(self.point_cloud, dtype=torch.float32), torch.tensor(self.label, dtype=torch.long)

# Create dataset and dataloader
label = 0  # Assuming '0' is the label for Coca-Cola can
dataset = PointCloudDataset(point_cloud, label)
dataloader = DataLoader(dataset, batch_size=1, shuffle=True)

# Instantiate and load the model
model = SimplePointNet(num_classes=2)  # Assuming binary classification
model = model.to('cuda' if torch.cuda.is_available() else 'cpu')


In [19]:
# Loss and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

# Training loop
model.train()
for epoch in range(10):  # Run for 10 epochs
    for points, labels in dataloader:
        points, labels = points.to('cuda'), labels.to('cuda')

        # Zero the parameter gradients
        optimizer.zero_grad()

        # Forward pass
        outputs = model(points)
        loss = criterion(outputs, labels)

        # Backward pass and optimize
        loss.backward()
        optimizer.step()

    print(f'Epoch [{epoch+1}/10], Loss: {loss.item():.4f}')

# Evaluation
model.eval()
with torch.no_grad():
    for points, labels in dataloader:
        points, labels = points.to('cuda'), labels.to('cuda')
        outputs = model(points)
        _, predicted = torch.max(outputs, 1)
        print(f'Predicted label: {predicted.item()}, True label: {labels.item()}')


Epoch [1/10], Loss: 0.5933
Epoch [2/10], Loss: 0.3440
Epoch [3/10], Loss: 0.1428
Epoch [4/10], Loss: 0.0267
Epoch [5/10], Loss: 0.0035
Epoch [6/10], Loss: 0.0008
Epoch [7/10], Loss: 0.0000
Epoch [8/10], Loss: 0.0000
Epoch [9/10], Loss: 0.0000
Epoch [10/10], Loss: 0.0000
Predicted label: 0, True label: 0


In [21]:
def load_point_cloud(file_path):
    # Load the GLB file as a scene
    scene = trimesh.load(file_path)

    # If the scene contains multiple geometries, you can merge them into a single mesh
    mesh = trimesh.util.concatenate(scene.dump()) if isinstance(scene, trimesh.Scene) else scene

    # Sample points from the surface of the mesh
    return mesh.sample(1024)

# Assuming you have a list of test file paths and their true labels
test_files = ['/content/coco_cola_can.glb', '/content/coca-cola_can__can_of_soda.glb']
true_labels = [0, 1]  # Assuming label 0 is for Coca-Cola can, 1 for other objects

# Prepare test dataset
test_data = [(load_point_cloud(file), label) for file, label in zip(test_files, true_labels)]


In [22]:
# Evaluation
model.eval()
correct = 0
total = 0

with torch.no_grad():
    for points, true_label in test_data:
        points = torch.tensor(points, dtype=torch.float32).unsqueeze(0)  # Add batch dimension
        points = points.to('cuda' if torch.cuda.is_available() else 'cpu')
        true_label = torch.tensor([true_label], dtype=torch.long).to('cuda' if torch.cuda.is_available() else 'cpu')

        # Forward pass
        outputs = model(points)
        _, predicted = torch.max(outputs, 1)

        total += true_label.size(0)
        correct += (predicted == true_label).sum().item()

accuracy = correct / total * 100
print(f'Accuracy: {accuracy:.2f}%')


Accuracy: 50.00%


In [23]:
# Save the trained model
torch.save(model.state_dict(), 'simple_pointnet_model.pth')
print("Model saved.")


Model saved.


In [33]:
import torch
import trimesh
import numpy as np
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader

# Define the PointNet model
class SimplePointNet(nn.Module):
    def __init__(self, num_classes=2):
        super(SimplePointNet, self).__init__()
        self.conv1 = nn.Conv1d(3, 64, 1)
        self.conv2 = nn.Conv1d(64, 128, 1)
        self.conv3 = nn.Conv1d(128, 1024, 1)
        self.fc1 = nn.Linear(1024, 512)
        self.fc2 = nn.Linear(512, 256)
        self.fc3 = nn.Linear(256, num_classes)
        self.relu = nn.ReLU()
        self.dropout = nn.Dropout(p=0.3)

    def forward(self, x):
        x = x.transpose(2, 1)  # Transpose to shape (B, 3, N)
        x = self.relu(self.conv1(x))
        x = self.relu(self.conv2(x))
        x = self.relu(self.conv3(x))
        x = torch.max(x, 2, keepdim=True)[0]  # Max pooling
        x = x.view(-1, 1024)
        x = self.relu(self.fc1(x))
        x = self.dropout(x)
        x = self.relu(self.fc2(x))
        x = self.fc3(x)
        return x

# Function to load and sample a point cloud from a GLB file
def load_point_cloud(file_path):
    # Load the GLB file as a scene
    scene = trimesh.load(file_path)

    # If the scene contains multiple geometries, merge them into a single mesh
    mesh = trimesh.util.concatenate(scene.dump()) if isinstance(scene, trimesh.Scene) else scene

    # Sample points from the surface of the mesh
    point_cloud = mesh.sample(1024)  # Adjust the number of points as needed
    return point_cloud

# Function to test the model on a given point cloud file path
def test_model(file_path, model, label_mapping):
    # Load point cloud
    point_cloud = load_point_cloud(file_path)

    # Convert point cloud to torch tensor
    points = torch.tensor(point_cloud, dtype=torch.float32).unsqueeze(0)  # Add batch dimension
    points = points.to('cuda' if torch.cuda.is_available() else 'cpu')

    # Forward pass through the model
    model.eval()
    with torch.no_grad():
        outputs = model(points)
        _, predicted = torch.max(outputs, 1)

    # Get the predicted label
    predicted_label = label_mapping[predicted.item()]
    print(f'Predicted label for {file_path}: {predicted_label}')

# Set up the label mapping
label_mapping = {0: "Coke", 1: "Unknown"}

# Load the trained model
model = SimplePointNet(num_classes=2)  # Adjust num_classes if you have more classes
model = model.to('cuda' if torch.cuda.is_available() else 'cpu')

# Load the model weights (replace with the actual path to your saved model weights)
# model.load_state_dict(torch.load('path/to/model_weights.pth'))

# Test the model on new data
test_files = ['/content/coco_cola_can.glb']  # Replace with your test file paths
model = "/content/simple_pointnet_model_new.pth"

for file_path in test_files:
    test_model(file_path, model, label_mapping)


AttributeError: 'str' object has no attribute 'eval'

finetuning


In [5]:
import trimesh
import numpy as np
import torch
from torch.utils.data import Dataset, DataLoader
import torch.nn as nn
import torch.optim as optim

In [27]:
class PointCloudDataset(Dataset):
    def __init__(self, file_paths, labels):
        self.file_paths = file_paths
        self.labels = labels

    def __len__(self):
        return len(self.file_paths)

    def __getitem__(self, idx):
        file_path = self.file_paths[idx]
        scene = trimesh.load(file_path)
        mesh = trimesh.util.concatenate(scene.dump()) if isinstance(scene, trimesh.Scene) else scene
        point_cloud = mesh.sample(1024)  # Sample 1024 points from the surface
        label = self.labels[idx]
        return torch.tensor(point_cloud, dtype=torch.float32), torch.tensor(label, dtype=torch.long)

# Example file paths and labels
file_paths = ['/content/coco_cola_can.glb', '/content/cokesoda.glb', '/content/coca-cola_can__can_of_soda.glb']  # Add paths to your GLB files
labels = [0, 0, 0]  # Assuming all are labeled as 0 (Coca-Cola cans)

# Create dataset and dataloader
dataset = PointCloudDataset(file_paths, labels)
dataloader = DataLoader(dataset, batch_size=1, shuffle=True)


In [28]:
class SimplePointNet(nn.Module):
    def __init__(self, num_classes=2):
        super(SimplePointNet, self).__init__()
        self.conv1 = nn.Conv1d(3, 64, 1)
        self.conv2 = nn.Conv1d(64, 128, 1)
        self.conv3 = nn.Conv1d(128, 1024, 1)
        self.fc1 = nn.Linear(1024, 512)
        self.fc2 = nn.Linear(512, 256)
        self.fc3 = nn.Linear(256, num_classes)
        self.relu = nn.ReLU()
        self.dropout = nn.Dropout(p=0.3)

    def forward(self, x):
        x = x.transpose(2, 1)  # Transpose to shape (B, 3, N)
        x = self.relu(self.conv1(x))
        x = self.relu(self.conv2(x))
        x = self.relu(self.conv3(x))
        x = torch.max(x, 2, keepdim=True)[0]  # Max pooling
        x = x.view(-1, 1024)
        x = self.relu(self.fc1(x))
        x = self.dropout(x)
        x = self.relu(self.fc2(x))
        x = self.fc3(x)
        return x

In [29]:
# Instantiate and load the model
model = SimplePointNet(num_classes=2)  # Assuming binary classification
model = model.to('cuda' if torch.cuda.is_available() else 'cpu')

# Loss and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

In [30]:
# Training loop
model.train()
for epoch in range(10):  # Run for 10 epochs
    for points, labels in dataloader:
        points, labels = points.to('cuda'), labels.to('cuda')

        # Zero the parameter gradients
        optimizer.zero_grad()

        # Forward pass
        outputs = model(points)
        loss = criterion(outputs, labels)

        # Backward pass and optimize
        loss.backward()
        optimizer.step()

    print(f'Epoch [{epoch+1}/10], Loss: {loss.item():.4f}')

Epoch [1/10], Loss: 0.1011
Epoch [2/10], Loss: 0.0000
Epoch [3/10], Loss: 0.0000
Epoch [4/10], Loss: 0.0000
Epoch [5/10], Loss: 0.0000
Epoch [6/10], Loss: 0.0000
Epoch [7/10], Loss: 0.0000
Epoch [8/10], Loss: 0.0000
Epoch [9/10], Loss: 0.0000
Epoch [10/10], Loss: 0.0000


In [31]:
model.eval()

SimplePointNet(
  (conv1): Conv1d(3, 64, kernel_size=(1,), stride=(1,))
  (conv2): Conv1d(64, 128, kernel_size=(1,), stride=(1,))
  (conv3): Conv1d(128, 1024, kernel_size=(1,), stride=(1,))
  (fc1): Linear(in_features=1024, out_features=512, bias=True)
  (fc2): Linear(in_features=512, out_features=256, bias=True)
  (fc3): Linear(in_features=256, out_features=2, bias=True)
  (relu): ReLU()
  (dropout): Dropout(p=0.3, inplace=False)
)

In [32]:
with torch.no_grad():
    for points, labels in dataloader:
        points, labels = points.to('cuda'), labels.to('cuda')
        outputs = model(points)
        _, predicted = torch.max(outputs, 1)
        print(f'Predicted label: {predicted.item()}, True label: {labels.item()}')

# Save the trained model
torch.save(model.state_dict(), 'simple_pointnet_model_new.pth')
print("Model saved.")

Predicted label: 0, True label: 0
Predicted label: 0, True label: 0
Predicted label: 0, True label: 0
Model saved.


In [37]:
import trimesh
import torch
import numpy as np

# Define the PointNet model class (make sure it matches the one used for training)
class SimplePointNet(nn.Module):
    def __init__(self, num_classes=2):
        super(SimplePointNet, self).__init__()
        self.conv1 = nn.Conv1d(3, 64, 1)
        self.conv2 = nn.Conv1d(64, 128, 1)
        self.conv3 = nn.Conv1d(128, 1024, 1)
        self.fc1 = nn.Linear(1024, 512)
        self.fc2 = nn.Linear(512, 256)
        self.fc3 = nn.Linear(256, num_classes)
        self.relu = nn.ReLU()
        self.dropout = nn.Dropout(p=0.3)

    def forward(self, x):
        x = x.transpose(2, 1)  # Transpose to shape (B, 3, N)
        x = self.relu(self.conv1(x))
        x = self.relu(self.conv2(x))
        x = self.relu(self.conv3(x))
        x = torch.max(x, 2, keepdim=True)[0]  # Max pooling
        x = x.view(-1, 1024)
        x = self.relu(self.fc1(x))
        x = self.dropout(x)
        x = self.relu(self.fc2(x))
        x = self.fc3(x)
        return x

# Function to load the model
def load_model(model_path):
    model = SimplePointNet(num_classes=2)
    model.load_state_dict(torch.load(model_path))
    model = model.to('cuda' if torch.cuda.is_available() else 'cpu')
    model.eval()
    return model

# Function to preprocess the new 3D object
def preprocess_3d_object(file_path):
    scene = trimesh.load(file_path)
    mesh = trimesh.util.concatenate(scene.dump()) if isinstance(scene, trimesh.Scene) else scene
    point_cloud = mesh.sample(1024)  # Sample 1024 points from the surface
    return torch.tensor(point_cloud, dtype=torch.float32).unsqueeze(0)  # Add batch dimension

# Function to predict the label of a new 3D object
def predict_new_object(model, file_path):
    point_cloud = preprocess_3d_object(file_path)
    point_cloud = point_cloud.to('cuda' if torch.cuda.is_available() else 'cpu')

    with torch.no_grad():
        outputs = model(point_cloud)
        _, predicted = torch.max(outputs, 1)

    return predicted.item()
# Set up the label mapping
label_mapping = {0: "Coke", 1: "Unknown"}

# Load the trained model
model_path = 'simple_pointnet_model_new.pth'
model = load_model(model_path)

# Test the new 3D object
new_object_path = '/content/cocacola_bottle.glb'  # Replace with your new GLB file path
predicted_label = predict_new_object(model, new_object_path)
print(f'Predicted label for the new 3D object: {label_mapping[predicted_label]}')


Predicted label for the new 3D object: Coke


color adding

In [46]:
import trimesh
import torch
import numpy as np
from torch.utils.data import Dataset, DataLoader

class PointCloudDatasetWithColor(Dataset):
    def __init__(self, file_paths, labels):
        self.file_paths = file_paths
        self.labels = labels

    def __len__(self):
        return len(self.file_paths)

    def __getitem__(self, idx):
        file_path = self.file_paths[idx]
        scene = trimesh.load(file_path)
        mesh = trimesh.util.concatenate(scene.dump()) if isinstance(scene, trimesh.Scene) else scene
        points = mesh.sample(1024)  # Sample 1024 points
        # Retrieve color information
        colors = np.zeros((1024, 3))  # Default to black if no colors are available
        if hasattr(mesh.visual, 'vertex_colors'):
            face_indices = mesh.face_adjacency_angles.sum(axis=1) < trimesh.constants.tol.merge
            colors = mesh.visual.face_colors[face_indices[:1024]]
        point_cloud = np.hstack((points, colors))  # Combine points and colors
        label = self.labels[idx]
        return torch.tensor(point_cloud, dtype=torch.float32), torch.tensor(label, dtype=torch.long)

# Example file paths and labels
file_paths = ['/content/coco_cola_can.glb']  # Add paths to your GLB files
labels = [0, 0, 0]  # Assuming all are labeled as 0 (Coca-Cola cans)

# Create dataset and dataloader
dataset = PointCloudDatasetWithColor(file_paths, labels)
dataloader = DataLoader(dataset, batch_size=1, shuffle=True)


In [63]:
import torch.nn as nn

class SimplePointNetWithColor(nn.Module):
    def __init__(self, num_classes=2):
        super(SimplePointNetWithColor, self).__init__()
        self.conv1 = nn.Conv1d(6, 64, 1)  # Input channels changed from 3 to 6
        self.conv2 = nn.Conv1d(64, 128, 1)
        self.conv3 = nn.Conv1d(128, 1024, 1)
        self.fc1 = nn.Linear(1024, 512)
        self.fc2 = nn.Linear(512, 256)
        self.fc3 = nn.Linear(256, num_classes)
        self.relu = nn.ReLU()
        self.dropout = nn.Dropout(p=0.3)

    def forward(self, x):
        x = x.transpose(2, 1)  # Transpose to shape (B, 6, N)
        x = self.relu(self.conv1(x))
        x = self.relu(self.conv2(x))
        x = self.relu(self.conv3(x))
        x = torch.max(x, 2, keepdim=True)[0]  # Max pooling
        x = x.view(-1, 1024)
        x = self.relu(self.fc1(x))
        x = self.dropout(x)
        x = self.relu(self.fc2(x))
        x = self.fc3(x)
        return x

# Instantiate and load the model
model = SimplePointNetWithColor(num_classes=2)  # Assuming binary classification
model = model.to('cuda' if torch.cuda.is_available() else 'cpu')


In [48]:
import torch.optim as optim

# Loss and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

# Training loop
model.train()
for epoch in range(10):  # Run for 10 epochs
    for points, labels in dataloader:
        points, labels = points.to('cuda'), labels.to('cuda')

        # Zero the parameter gradients
        optimizer.zero_grad()

        # Forward pass
        outputs = model(points)
        loss = criterion(outputs, labels)

        # Backward pass and optimize
        loss.backward()
        optimizer.step()

    print(f'Epoch [{epoch+1}/10], Loss: {loss.item():.4f}')

# Save the trained model
torch.save(model.state_dict(), 'simple_pointnet_with_color_model.pth')
print("Model saved.")


Epoch [1/10], Loss: 0.7016
Epoch [2/10], Loss: 0.4517
Epoch [3/10], Loss: 0.2809
Epoch [4/10], Loss: 0.0845
Epoch [5/10], Loss: 0.0210
Epoch [6/10], Loss: 0.0040
Epoch [7/10], Loss: 0.0031
Epoch [8/10], Loss: 0.0000
Epoch [9/10], Loss: 0.0000
Epoch [10/10], Loss: 0.0000
Model saved.


In [68]:
# Function to preprocess the new 3D object with color
def preprocess_3d_object_with_color(file_path):
    scene = trimesh.load(file_path)
    mesh = trimesh.util.concatenate(scene.dump()) if isinstance(scene, trimesh.Scene) else scene
    points = mesh.sample(1024)  # Sample 1024 points
    # Retrieve color information
    colors = np.zeros((1024, 3))  # Default to black if no colors are available
    if hasattr(mesh.visual, 'vertex_colors'):
        face_indices = mesh.face_adjacency_angles.sum(axis=1) < trimesh.constants.tol.merge
        colors = mesh.visual.face_colors[face_indices[:1024]]
    point_cloud = np.hstack((points, colors))  # Combine points and colors
    return torch.tensor(point_cloud, dtype=torch.float32).unsqueeze(0)  # Add batch dimension

# Function to load the model
def load_model_with_color(model_path):
    model = SimplePointNetWithColor(num_classes=2)
    model.load_state_dict(torch.load(model_path))
    model = model.to('cuda' if torch.cuda.is_available() else 'cpu')
    model.eval()
    return model

# Function to predict the label of a new 3D object with color and confidence threshold
def predict_new_object_with_confidence(model, file_path, threshold=0.99):
    point_cloud = preprocess_3d_object_with_color(file_path)
    point_cloud = point_cloud.to('cuda' if torch.cuda.is_available() else 'cpu')

    with torch.no_grad():
        outputs = model(point_cloud)
        probabilities = torch.softmax(outputs, dim=1)
        confidence, predicted = torch.max(probabilities, 1)

    confidence = confidence.item()
    predicted_label = predicted.item()

    if confidence >= threshold:
        return predicted_label, confidence
    else:
        return 1, confidence

# Load the trained model with color support
model_path = '/content/simple_pointnet_with_color_model.pth'
model = load_model_with_color(model_path)
label_mapping = {0: "Coke", 1: "Unknown"}

# Test the new 3D object with color information and confidence threshold
new_object_path = '/content/pepsi_can.glb'  # Replace with your new GLB file path
predicted_label, confidence = predict_new_object_with_confidence(model, new_object_path)
print(f'Predicted label for the new 3D object: label: {label_mapping[predicted_label]}, Confidence: {confidence:.4f}')


Predicted label for the new 3D object: label: Unknown, Confidence: 0.8290
